In [46]:
### UTILITIES
from keras.utils import to_categorical
import pickle
import sys
from matplotlib import pyplot as plt
import tensorflow as tf
import numpy as np
from keras.datasets import cifar10
from PIL import Image
import os
import numpy as np


"""
def load_images_from_folder(folder_path, image_shape):
    images = []
    labels = []

    for label, class_folder in enumerate(sorted(os.listdir(folder_path))):
        class_path = os.path.join(folder_path, class_folder)
        for filename in os.listdir(class_path):
            img_path = os.path.join(class_path, filename)
            img = load_image(img_path, image_shape)
            images.append(img)
            labels.append(label)

    images = np.array(images)
    labels = np.array(labels)
    return images, labels

import numpy as np
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

def load_dataset(batch_size, num_classes, epochs):
    # Load CIFAR-10 dataset
    (x_train_full, y_train_full), (x_test, y_test) = cifar10.load_data()

    # Normalize images (convert to float32 and scale to [0,1])
    x_train_full = x_train_full.astype('float32') / 255.0
    x_test = x_test.astype('float32') / 255.0

    # Convert labels to one-hot encoding
    y_train_full = to_categorical(y_train_full, num_classes)
    y_test = to_categorical(y_test, num_classes)

    # Split training data into 40,000 for training and 10,000 for validation
    x_train, x_val, y_train, y_val = train_test_split(
        x_train_full, y_train_full, test_size=0.2, random_state=42
    )

    dataset = {
        'batch_size': batch_size,
        'num_classes': num_classes,
        'epochs': epochs,
        'x_train': x_train,
        'y_train': y_train,
        'x_val': x_val,
        'y_val': y_val,
        'x_test': x_test,  
        'y_test': y_test
    }

    return dataset
"""

def save_network(network):
    #object_file = open(network.name + '.obj', 'wb')
    #pickle.dump(network, object_file)
    #tf.keras.models.save_model(network, network.name)

    model_path = network.name + '_model.h5'
    tf.keras.models.save_model(network.model, model_path)

    # Save the rest of the network information
    network_info = {
        'name': network.name,
        'block_list': network.block_list,
        'fitness': network.fitness
    }
    network_info_path = network.name + '_info.pkl'
    with open(network_info_path, 'wb') as info_file:
        pickle.dump(network_info, info_file)


def load_network(name):
    model_path = name + '_model.h5'
    loaded_model = tf.keras.models.load_model(model_path)

    # Load the network information
    info_path = name + '_info.pkl'
    with open(info_path, 'rb') as info_file:
        network_info = pickle.load(info_file)

    # Create a new Network instance
    loaded_network = Network(0)  # Update with appropriate 'it' value

    # Set the attributes of the loaded network
    loaded_network.name = network_info['name']
    loaded_network.block_list = network_info['block_list']
    loaded_network.fitness = network_info['fitness']
    loaded_network.model = loaded_model

    return loaded_network



def order_indexes(self):
    i = 0
    for block in self.block_list:
        block.index = i
        i += 1


def plot_training(history):                                           # plot diagnostic learning curves
    plt.figure(figsize=[8, 6])  # accuracy curves
    plt.plot(history.history['accuracy'], 'r', linewidth=3.0)
    plt.plot(history.history['val_accuracy'], 'b', linewidth=3.0)  # <-- Change 'val_acc' to 'val_accuracy'
    plt.legend(['Training Accuracy', 'Validation Accuracy'], fontsize=18)
    plt.xlabel('Epochs ', fontsize=16)
    plt.ylabel('Accuracy', fontsize=16)
    plt.title('Accuracy Curves', fontsize=16)

    filename = sys.argv[0].split('/')[-1]
    plt.savefig(filename + '_acc_plot.png')
    plt.close()



def plot_statistics(stats):
    plt.figure(figsize=[8, 6])											# fitness curves
    plt.plot([s[0] for s in stats], 'r', linewidth=3.0)
    plt.plot([stats[0][0]] * len(stats), 'b', linewidth=3.0)
    plt.legend(['BestFitness', 'InitialFitness'], fontsize=18)
    plt.xlabel('Generations', fontsize=16)
    plt.ylabel('FitnessValue', fontsize=16)
    plt.title('Fitness Curve', fontsize=16)
    filename = sys.argv[0].split('/')[-1]
    plt.savefig(filename + '_fitness_plot.png')

    plt.figure(figsize=[8, 6])											# parameters curves
    plt.plot([s[1] for s in stats], 'r', linewidth=3.0)
    plt.plot([stats[0][1]] * len(stats), 'b', linewidth=3.0)
    plt.legend(['BestParamsNum', 'InitialParamsNum'], fontsize=18)
    plt.xlabel('Generations', fontsize=16)
    plt.ylabel('ParamsNum', fontsize=16)
    plt.title('Parameters Curve', fontsize=16)
    filename = sys.argv[0].split('/')[-1]
    plt.savefig(filename + '_params_plot.png')
    plt.close()


In [47]:
#(Block number,position)
# initial - 0, # middle -1, Final -2

# INOUT
import os
def compute_parent(dataset):
    if os.path.isfile('parent_0.h5'):
        daddy = load_network('parent_0')
        model = tf.keras.models.load_model('parent_0.h5')
        print("Loading parent_0")
        print("SUMMARY OF", daddy.name)
        print(model.summary())
        print("FITNESS:", daddy.fitness)
        return daddy

    daddy = Network(0)
    
    
    #INI BLOCK
    layerList1 = [
        Convolutional(filters=32, filter_size=(3, 3), stride_size=(1, 1), padding='same',
                      input_shape=dataset['x_train'].shape[1:]),
        Pooling(pool_size=(2, 2), stride_size=(2, 2), padding='same')
    ]
    layerList2 = [
        Dropout(0.25)
    ]
    daddy.block_list.append(Block(0, 0, layerList1, layerList2))
    
    #MIDDLE BLOCK 1
    layerList1 = [
        Convolutional(filters=32, filter_size=(3, 3), stride_size=(1, 1), padding='same',
                      input_shape=dataset['x_train'].shape[1:]),
        Pooling(pool_size=(2, 2), stride_size=(2, 2), padding='same')
    ]
    layerList2 = [
        Dropout(0.25)
    ]
    daddy.block_list.append(Block(1, 1, layerList1, layerList2))

    #FULLY CONNECTED LAYER
    layerList1 = [
        FullyConnected(units=128, num_classes=dataset['num_classes'])
    ]
    layerList2 = []
    daddy.block_list.append(Block(2, 5, layerList1, layerList2))
    
    

    model = daddy.build_model()
    print("Type of model_final:", type(model))
    daddy.train_and_evaluate(model, dataset)
    return daddy

In [48]:
# NETWORK
import tensorflow as tf
import os
import pickle
from keras.callbacks import Callback
from keras.models import Sequential
from random import randint, choice
from copy import deepcopy


class Network:
    __slots__ = ('name', 'block_list', 'fitness', 'model')

    def __init__(self, it):
        self.name = 'parent_' + str(it) if it == 0 else 'net_' + str(it)
        self.block_list = []
        self.fitness = None
        self.model = None

    """def build_model(self):
        model = Sequential()                                # create model
        for block in self.block_list:
            for layer in block.get_layers():                # build model
                try:
                    layer.build_layer(model)
                except:
                    print("\nINDIVIDUAL ABORTED, CREATING A NEW ONE\n")
                    return -1
        return model"""
    def build_model(self):
        model = Sequential()              
        print("The block is:")
        print(self.block_list)                 # create model
        for block in self.block_list:
            #print("Building block type:", block.type)
            #print("TOTAL :::")
            #print(block.get_layer_name())
            for layer in block.get_layers():                # build model
                #print("Adding layer:", layer.name)
                try:
                    layer.build_layer(model)
                    print("Layer added successfully.")
                except Exception as e:
                    print("Error occurred while adding layer:", e)
                    print("Returning None.")
                    return -1
        print("Model successfully built.")
        return model

    def train_and_evaluate(self, model, dataset):
        print("Training", self.name)
        model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
        try:
            history = model.fit(dataset['x_train'],
                                dataset['y_train'],
                                batch_size=dataset['batch_size'],
                                epochs=dataset['epochs'],
                                validation_data=(dataset['x_val'], dataset['y_val']),
                                shuffle=True)
        except Exception as e:
            print("An error occurred during model training:", e)
            return -1
            # You can choose to handle the error in a specific way here, like logging it or taking corrective actions.


        # Extract metrics from the training history
        training_loss = history.history['loss'][-1]
        training_accuracy = history.history['accuracy'][-1]
        validation_loss = history.history['val_loss'][-1]
        validation_accuracy = history.history['val_accuracy'][-1]

        print(model.summary())
        x= input("Do you want to continue ?")
        # Additional metrics (you can customize this based on your needs)
        classification_error_rate = 1.0 - validation_accuracy

        self.model = model  # Save the model
        self.fitness = validation_loss  # Use validation loss as fitness

        # Print metrics
        print("SUMMARY OF", self.name)
        print("Training Loss:", training_loss)
        print("Training Accuracy:", training_accuracy)
        print("Validation Loss:", validation_loss)
        print("Validation Accuracy:", validation_accuracy)
        print("Classification Error Rate:", classification_error_rate)

        tf.keras.models.save_model(model, self.name + '.h5')         # save model
        #model.save(self.name + '.h5')                       # save model
        save_network(self)                                  # save topology, model and fitness

    def asexual_reproduction(self, it, dataset):

        # if the individual already exists, just load it
        if os.path.isfile('net_' + str(it) + '.h5'):
            print("\n-------------------------------------")
            print("Loading individual net_" + str(it))
            print("--------------------------------------\n")
            individual = load_network('net_' + str(it))
            model = tf.keras.models.load_model(individual.name + '.h5')
            print("SUMMARY OF", individual.name)
            print(model.summary())
            print("FITNESS: ", individual.fitness)
            return individual

        # otherwise, create the individual by mutating the parent
        individual = Network(it)

        print("\n-------------------------------------")
        print("\nCreating individual", individual.name)
        print("--------------------------------------\n")

        individual.block_list = deepcopy(self.block_list)           # copy the layer list from parent

        print("----->Strong Mutation")
        individual.block_mutation(dataset)                          # mutate a block
        individual.layer_mutation(dataset)                          # mutate a layer
        individual.parameters_mutation()                            # mutate some parameters

        model = individual.build_model()
        
        if model == -1:
            return self.asexual_reproduction(it, dataset)
        
        if(individual.train_and_evaluate(model, dataset)==-1):
            return self.asexual_reproduction(it, dataset)
        else:
            return individual
            

    def block_mutation(self, dataset):
        try:
            print("Block Mutation")

            print([(block.index, block.type) for block in self.block_list])

            # block list containing all the blocks with type = 1
            bl = [block.index for block in self.block_list if block.type == 1]

            if len(bl) == 0:
                print("Creating a new block with two Convolutional layers and a Pooling layer")
                self.block_list[1].index = 2
                layerList1 = [
                    Convolutional(filters=pow(2, randint(5, 8)),
                                  filter_size=(3, 3),
                                  stride_size=(1, 1),
                                  padding='same',
                                  input_shape=dataset['x_train'].shape[1:]),
                    Convolutional(filters=pow(2, randint(5, 8)),
                                  filter_size=(3, 3),
                                  stride_size=(1, 1),
                                  padding='same',
                                  input_shape=dataset['x_train'].shape[1:])
                ]
                layerList2 = [
                    Pooling(pool_size=(2, 2),
                            stride_size=(2, 2),
                            padding='same')
                ]
                b = Block(1, 1, layerList1, layerList2)
                self.block_list.insert(1, b)
                return

            block_idx = randint(1, max(bl))         # pick a random block among all the blocks with type = 1
            block_type_idx = randint(0, 1)          # 1 -> Conv2D; 0 -> Pooling or Dropout
            mutation_type = randint(0, 1)           # 1 -> remove; 0 -> add

            # list of layers of the selected block
            layerList = self.block_list[block_idx].layerList1 if block_type_idx else self.block_list[block_idx].layerList2
            length = len(layerList)

            if mutation_type:                                       # remove
                if length == 1:
                    del self.block_list[block_idx]
                elif block_type_idx:
                    pos = randint(0, length - 1)
                    print("Removing a Conv2D layer at", pos)
                    del layerList[pos]
                else:
                    pos = randint(0, length - 1)
                    print("Removing a Pooling/Dropout layer at", pos)
                    del layerList[pos]
            else:                                                   # add
                if block_type_idx:
                    print("Inserting a Convolutional layer")
                    layer = Convolutional(filters=pow(2, randint(5, 8)),
                                          filter_size=(3, 3),
                                          stride_size=(1, 1),
                                          padding='same',
                                          input_shape=dataset['x_train'].shape[1:])
                    layerList.insert(randint(0, length - 1), layer)
                else:
                    if randint(0, 1):                               # 1 -> Pooling; 0 -> Dropout
                        print("Inserting a Pooling layer")
                        layer = Pooling(pool_size=(2, 2),
                                        stride_size=(2, 2),
                                        padding='same')
                        layerList.insert(randint(0, length - 1), layer)
                    else:
                        print("Inserting a Dropout layer")
                        rate = choice([0.15, 0.25, 0.35, 0.50])
                        layer = Dropout(rate=rate)
                        layerList.insert(randint(0, length - 1), layer)
        except Exception as e:
            print(f"An error occurred during block mutation: {e}")
            return None

                    
                    
                    
                    
                    

    """def layer_mutation(self, dataset):
        print("Layer Mutation")

        # pick a random block among all the blocks with type = 1
        bl = [block.index for block in self.block_list if block.type == 1]

        if len(bl) == 0:
            return

        block_idx = randint(1, max(bl))
        block_type_idx = randint(0, 1)      # 1 -> Conv2D; 0 -> Pooling or Dropout

        # list of layers of the selected block
        layerList = self.block_list[block_idx].layerList1 if block_type_idx else self.block_list[block_idx].layerList2

        if len(layerList) == 0:
            if block_type_idx:
                layer = Convolutional(filters=pow(2, randint(5, 8)),
                                      filter_size=(3, 3),
                                      stride_size=(1, 1),
                                      padding='same',
                                      input_shape=dataset['x_train'].shape[1:])
                self.block_list[block_idx].layerList1.append(layer)
                return
            else:
                layer = Pooling(pool_size=(2, 2),
                                stride_size=(2, 2),
                                padding='same')
                self.block_list[block_idx].layerList2.append(layer)

        idx = randint(0, len(layerList) - 1)
        layer = layerList[idx]

        if layer.name == 'Conv2D':
            print("Splitting Conv2D layer at index", idx)
            layer.filters = int(layer.filters * 0.5)
            layerList.insert(idx, deepcopy(layer))
        elif layer.name == 'MaxPooling2D' or layer.name == 'AveragePooling2D':
            print("Changing Pooling layer at index", idx, "with Conv2D layer")
            del layerList[idx]
            conv_layer = Convolutional(filters=pow(2, randint(5, 8)),
                                       filter_size=(3, 3),
                                       stride_size=(2, 2),
                                       padding=layer.padding,
                                       input_shape=dataset['x_train'].shape[1:])
            layerList.insert(idx, conv_layer)"""
    
    def layer_mutation(self, dataset):
        print("Layer Mutation")

        # Determine the maximum number of layers that can be added or removed
        max_layers_to_add = 16 - sum(len(block.layerList1) + len(block.layerList2) for block in self.block_list)
        max_layers_to_remove = sum(len(block.layerList1) + len(block.layerList2) - 1 for block in self.block_list)

        if max_layers_to_add == 0 and max_layers_to_remove == 0:
            return

        # Pick a random block among all the blocks with type = 1
        bl = [block.index for block in self.block_list if block.type == 1]

        if len(bl) == 0:
            return

        block_idx = randint(1, max(bl))
        block_type_idx = randint(0, 1)      # 1 -> Conv2D; 0 -> Pooling or Dropout

        # List of layers of the selected block
        layerList = self.block_list[block_idx].layerList1 if block_type_idx else self.block_list[block_idx].layerList2

        if len(layerList) == 0:
            if block_type_idx:
                layer = Convolutional(filters=pow(2, randint(5, 8)),
                                      filter_size=(3, 3),
                                      stride_size=(1, 1),
                                      padding='same',
                                      input_shape=dataset['x_train'].shape[1:])
                self.block_list[block_idx].layerList1.append(layer)
            else:
                layer = Pooling(pool_size=(2, 2),
                                stride_size=(2, 2),
                                padding='same')
                self.block_list[block_idx].layerList2.append(layer)
        else:
            # Randomly choose whether to add or remove a layer
            add_layer = bool(randint(0, 1))

            if add_layer and max_layers_to_add > 0:
                # Add a layer
                layer = self.create_random_layer(dataset)
                layerList.insert(randint(0, len(layerList)), layer)
            elif not add_layer and max_layers_to_remove > 0:
                # Remove a layer
                idx = randint(0, len(layerList) - 1)
                del layerList[idx]

        # Ensure the total number of layers in the block doesn't exceed 16
        if len(self.block_list[block_idx].layerList1) + len(self.block_list[block_idx].layerList2) > 16:
            # Remove a random layer to maintain the total count of 16 layers
            block_layerList = self.block_list[block_idx].layerList1 if block_type_idx else self.block_list[block_idx].layerList2
            del block_layerList[randint(0, len(block_layerList) - 1)]

    def create_random_layer(self, dataset):
        # Create a random layer (Conv2D or Pooling)
        if randint(0, 1):
            # Conv2D layer
            return Convolutional(filters=pow(2, randint(5, 8)),
                                 filter_size=(3, 3),
                                 stride_size=(1, 1),
                                 padding='same',
                                 input_shape=dataset['x_train'].shape[1:])
        else:
            # Pooling layer
            return Pooling(pool_size=(2, 2),
                           stride_size=(2, 2),
                           padding='same')

            
            
            
            
            
            
            
            

    def parameters_mutation(self):
        print("Parameters Mutation")
        for block in self.block_list:
            for layer in block.get_layers():
                if randint(0, 1):
                    layer.mutate_parameters()

    def save_network_info(self, info_filename):
        network_info = {
            'name': self.name,
            'block_list': self.block_list,
            'fitness': self.fitness
        }

        with open(info_filename, 'wb') as info_file:
            pickle.dump(network_info, info_file)

    def load_network_info(self, info_filename):
        with open(info_filename, 'rb') as info_file:
            network_info = pickle.load(info_file)

        self.name = network_info['name']
        self.block_list = network_info['block_list']
        self.fitness = network_info['fitness']

    def save_model(self, model_filename):
        self.model.save(model_filename)

    def load_model(self, model_filename):
        self.model = tf.keras.models.load_model(model_filename)

    def save_network(self, network_info_filename, model_filename):
        # Save non-model attributes
        self.save_network_info(network_info_filename)

        # Save the model separately
        self.save_model(model_filename)

    def load_network(self, network_info_filename, model_filename):
        # Load non-model attributes
        self.load_network_info(network_info_filename)

        # Load the model separately
        self.load_model(model_filename)


In [49]:
# TOPOLOGY

import keras.layers
from random import randint


class Block:
	__slots__ = ('type', 'index', 'layerList1', 'layerList2')

	def __init__(self, type, index, layerList1, layerList2):
		self.type = type										# 0 -> initial layer; 1 -> mid layers; 2 -> final layer
		self.index = index										# block index among all the blocks
		self.layerList1 = layerList1							# Convolutional layers
		self.layerList2 = layerList2							# Pooling and Dropout layers

	def get_layers(self):
		return self.layerList1 + self.layerList2

	def get_size(self):
		return len(self.get_layers())


class Convolutional:
	# __slots__ = ('name', 'filters', 'padding', 'filter_size', 'stride_size', 'input_shape')

	def __init__(self, filters, padding, filter_size, stride_size, input_shape):
		self.name = 'Conv2D'
		self.filters = filters
		self.padding = padding
		self.filter_size = filter_size
		self.stride_size = stride_size
		self.input_shape = input_shape

	def build_layer(self, model):
		try:
			model.add(keras.layers.Conv2D(filters=self.filters,
											kernel_size=self.filter_size,
											strides=self.stride_size,
											padding=self.padding,
											activation='relu',
											kernel_initializer='he_uniform',
											input_shape=self.input_shape))
		except ValueError as e:
			print("Error occurred while adding layer:", e)
			print("Skipping current architecture.")
			return  # Skip adding this layer
	def mutate_parameters(self):
		mutation = randint(0, 2)  # Adjusted the number of mutations
		print("Mutating", self.name, "layer:")
		if mutation == 0 and self.filters >= 64:  # Adjusted the filter reduction threshold
			print("-->changed self.filters from ", self.filters, " ", end="")
			self.filters = int(self.filters / 2)
			print("to ", self.filters)
		elif mutation == 1 and self.filters <= 256:  # Adjusted the filter increase threshold
			print("-->changed self.filters from ", self.filters, " ", end="")
			self.filters *= 2
			print("to ", self.filters)
		elif mutation == 2:
			if self.padding == 'valid':
				print("-->changed self.padding from ", self.padding, " ", end="")
				self.padding = 'same'
				print("to ", self.padding)
			else:
				print("-->changed self.padding from ", self.padding, " ", end="")
				self.padding = 'valid'
				print("to ", self.padding)

        

        


	"""def mutate_parameters(self):
		mutation = randint(0, 4)
		print("Mutating", self.name, "layer:")
		if mutation == 0 and self.filters >= 32:
			print("-->changed self.filters from ", self.filters, " ", end="")
			self.filters = int(self.filters / 2)
			print("to ", self.filters)
		elif mutation == 1 and self.filters >= 32:
			print("-->changed self.filters from ", self.filters, " ", end="")
			self.filters = int(self.filters / 2)
			print("to ", self.filters)
		elif mutation == 2 and self.filters <= 512:
			print("-->changed self.filters from ", self.filters, " ", end="")
			self.filters *= 2
			print("to ", self.filters)
		elif mutation == 3 and self.filters <= 512:
			print("-->changed self.filters from ", self.filters, " ", end="")
			self.filters *= 2
			print("to ", self.filters)
		elif mutation == 4:
			if self.padding == 'valid':
				print("-->changed self.padding from ", self.padding, " ", end="")
				self.padding = 'same'
				print("to ", self.padding)
			else:
				print("-->changed self.padding from ", self.padding, " ", end="")
				self.padding = 'valid'
				print("to ", self.padding)
"""
    

'''
elif mutation is 4:
	print("changed self.stride_size from ", self.stride_size, " ", end="")
	self.stride_size = (self.stride_size[0] + 1, self.stride_size[1] + 1)
	print("to ", self.stride_size, " and ", end="")
	print("changed self.stride_size from ", self.stride_size, " ", end="")
	self.stride_size = (self.stride_size[0] + 1, self.stride_size[1] + 1)
	print("to ", self.stride_size)
'''


class Pooling:
	__slots__ = ('name', 'pool_size', 'stride_size', 'padding')

	def __init__(self, pool_size, stride_size, padding):
		self.name = 'MaxPooling2D'
		self.pool_size = pool_size
		self.stride_size = stride_size
		self.padding = padding

	def build_layer(self, model):
		if self.name == 'MaxPooling2D':
			model.add(keras.layers.MaxPooling2D(self.pool_size, self.stride_size, self.padding))
		elif self.name == 'AveragePooling2D':
			model.add(keras.layers.AveragePooling2D(self.pool_size, self.stride_size, self.padding))

	def mutate_parameters(self):
		print("Mutating", self.name, "layer:")
		mutation = randint(0, 1)
		if mutation == 0:
			if self.padding == 'valid':
				print("-->changed self.padding from ", self.padding, " ", end="")
				self.padding = 'same'
				print("to ", self.padding)
			else:
				print("-->changed self.padding from ", self.padding, " ", end="")
				self.padding = 'valid'
				print("to ", self.padding)
		elif mutation == 1:
			if self.name == 'MaxPooling2D':
				print("-->changed self.name from ", self.name, " ", end="")
				self.name = 'AveragePooling2D'
				print("to ", self.name)
			else:
				print("-->changed self.name from ", self.name, " ", end="")
				self.name = 'MaxPooling2D'
				print("to ", self.name)


'''
if mutation is 0:
	print("changed self.stride_size from ", self.stride_size, " ", end="")
	self.stride_size = (self.stride_size[0] + 1, self.stride_size[1] + 1)
	print("to ", self.stride_size)
'''


class FullyConnected:
	__slots__ = ('name', 'units', 'num_classes')

	def __init__(self, units, num_classes):
		self.name = "FullyConnected"
		self.units = units
		self.num_classes = num_classes

	def build_layer(self, model):
		model.add(keras.layers.Flatten())
		model.add(keras.layers.Dense(self.units, activation='relu', kernel_initializer='he_uniform'))
		model.add(keras.layers.Dense(self.num_classes, activation='softmax'))

	def mutate_parameters(self):
		print("Mutating", self.name, "layer:")
		mutation = randint(0, 2)
		if mutation == 0:
			print("-->changed self.units from ", self.units, " ", end="")
			self.units *= 2
			print("to ", self.units)
		elif mutation == 1:
			print("-->changed self.units from ", self.units, " ", end="")
			self.units *= 2
			print("to ", self.units)
		elif mutation == 2:
			print("-->changed self.units from ", self.units, " ", end="")
			self.units /= 2
			print("to ", self.units)


'''
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(self.num_classes, activation='softmax'))
'''


class Dropout:
	__slots__ = ('name', 'rate')

	def __init__(self, rate):
		self.name = "Dropout"
		self.rate = rate

	def build_layer(self, model):
		model.add(keras.layers.Dropout(self.rate))

	def mutate_parameters(self):
		print("Mutating", self.name, "layer:")
		mutation = randint(0, 3)
		if mutation == 0 and self.rate <= 0.85:
			print("-->changed self.rate from ", self.rate, " ", end="")
			self.rate = self.rate + 0.10
			print("to ", self.rate)
		elif mutation == 1 and self.rate <= 0.90:
			print("-->changed self.rate from ", self.rate, " ", end="")
			self.rate = self.rate + 0.05
			print("to ", self.rate)
		elif mutation == 2 and self.rate >= 0.15:
			print("-->changed self.rate from ", self.rate, " ", end="")
			self.rate = self.rate - 0.10
			print("to ", self.rate)
		elif mutation == 3 and self.rate >= 0.10:
			print("-->changed self.rate from ", self.rate, " ", end="")
			self.rate = self.rate - 0.05
			print("to ", self.rate)

class FlattenLayer:
    def __init__(self):
        self.name = 'Flatten'

    def build_layer(self, model):
        model.add(keras.layers.Flatten())

    def mutate_parameters(self):
        # The Flatten layer does not have any parameters to mutate
        pass


In [50]:
import tensorflow as tf
tf.compat.v1.enable_eager_execution()
import os
from copy import deepcopy
from random import sample
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)      # suppress messages from Tensorflow
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'


def initialize_population(population_size, dataset):
    print("----->Initializing Population")
    daddy = compute_parent(dataset)                                 # load parent from input
    population = [daddy]
    for it in range(1, population_size):
        population.append(daddy.asexual_reproduction(it, dataset))

    # sort population on ascending order based on fitness
    return sorted(population, key=lambda cnn: cnn.fitness)


def selection(k, population, num_population):
    if k == 0:                                              # elitism selection
        print("----->Elitism selection")
        return population[0], population[1]
    elif k == 1:                                            # tournament selection
        print("----->Tournament selection")
        i = randint(0, num_population - 1)
        j = i
        while j < num_population - 1:
            j += 1
            if randint(1, 100) <= 50:
                return population[i], population[j]
        return population[i], population[0]
    else:                                                   # proportionate selection
        print("----->Proportionate selection")
        cum_sum = 0
        for i in range(num_population):
            cum_sum += population[i].fitness
        perc_range = []
        for i in range(num_population):
            count = 100 - int(100 * population[i].fitness / cum_sum)
            for j in range(count):
                perc_range.append(i)
        i, j = sample(range(1, len(perc_range)), 2)
        while i == j:
            i, j = sample(range(1, len(perc_range)), 2)
        return population[perc_range[i]], population[perc_range[j]]


def crossover(parent1, parent2, it):
    print("----->Crossover")
    child = Network(it)

    first, second = None, None
    if randint(0, 1):
        first = parent1
        second = parent2
    else:
        first = parent2
        second = parent1

    child.block_list = deepcopy(first.block_list[:randint(1, len(first.block_list) - 1)]) \
                       + deepcopy(second.block_list[randint(1, len(second.block_list) - 1):])

    order_indexes(child)                            # order the indexes of the blocks

    return child


def genetic_algorithm(num_population, num_generation, num_offspring, dataset, early_stopping_generations=3):
    print("Genetic Algorithm")

    population = initialize_population(num_population, dataset)

    print("\n-------------------------------------")
    print("Initial Population:")
    for cnn in population:
        print(cnn.name, ': ', cnn.fitness)
    print("--------------------------------------\n")

    # for printing statistics about fitness and the number of parameters of the best individual
    stats = [(population[0].fitness, population[0].model.count_params())]

    # Initialize a variable to keep track of consecutive generations with the same best fitness
    consecutive_same_fitness = 0

    for gen in range(1, num_generation + 1):
        '''
            k is the selection parameter:
                k = 0 -> elitism selection
                k = 1 -> tournament selection
                k = 2 -> proportionate selection
        '''
        k = randint(0, 2)

        print("\n------------------------------------")
        print("Generation -----------------------------------------------------------------------------------", gen)
        print("-------------------------------------")

        for c in range(num_offspring):

            print("\nCreating Child", c)

            parent1, parent2 = selection(k, population, num_population)                 # selection
            print("Selected", parent1.name, "and", parent2.name, "for reproduction")

            child = crossover(parent1, parent2, c + num_population)                     # crossover
            print("Child has been created")

            print("----->Soft Mutation")
            child.layer_mutation(dataset)                                               # mutation
            child.parameters_mutation()
            print("Child has been mutated")

            model = child.build_model()                                                 # evaluation

            while model == -1:
                child = crossover(parent1, parent2, c + num_population)
                child.block_mutation(dataset)
                child.layer_mutation(dataset)
                child.parameters_mutation()
                model = child.build_model()

            child.train_and_evaluate(model, dataset)

            if child.fitness < population[-1].fitness:                                  # evolve population
                print("----->Evolution: Child", child.name, "with fitness", child.fitness, "replaces parent ", end="")
                print(population[-1].name, "with fitness", population[-1].fitness)
                name = population[-1].name

                child.save_network("child_model_info.pkl", "child_model.h5")
                population[-1].load_network("child_model_info.pkl", "child_model.h5")

                population[-1].name = name
                population = sorted(population, key=lambda net: net.fitness)
            else:
                print("----->Evolution: Child", child.name, "with fitness", child.fitness, "is discarded")
        
        if gen >= 3 and all(population[i].fitness == population[i + 1].fitness for i in range(-3, -1)):
            consecutive_same_fitness += 1
            print(f"Consecutive generations with the same best fitness: {consecutive_same_fitness}")
        if consecutive_same_fitness >= 3:
            print("Stopping the algorithm as the best fitness has remained the same for the last 3 generations.")
            break
    else:
        consecutive_same_fitness = 0
        
       #Check if the best fitness has remained the same for the last early_stopping_generations generations
        if all(population[i].fitness == population[i + 1].fitness for i in range(-early_stopping_generations, -1)):
            consecutive_same_fitness += 1
            print(f"Consecutive generations with the same best fitness: {consecutive_same_fitness}")
            if consecutive_same_fitness == early_stopping_generations:
                print(f"Stopping the algorithm as the best fitness has remained the same for {early_stopping_generations} generations.")
        else:
            consecutive_same_fitness = 0
        stats.append((population[0].fitness, population[0].model.count_params()))

    print("\n\n-------------------------------------")
    print("Final Population")
    print("-------------------------------------\n")
    for cnn in population:
        print(cnn.name, ': ', cnn.fitness)

    print("\n-------------------------------------")
    print("Stats")
    for i in range(len(stats)):
        print("Best individual at generation", i + 1, "has fitness", stats[i][0], "and parameters", stats[i][1])
    print("-------------------------------------\n")

    # plot the fitness and the number of parameters of the best individual at each iteration
    plot_statistics(stats)

    return population[0]



def main(): 
        
        #with strategy.scope():
        #from tensorflow.python.client import device_lib
        #print(device_lib.list_local_devices())
        #batch_size = 8
        #batch_size = batch_size * strategy.num_replicas_in_sync
        batch_size = 32                       # the number of training examples in one forward/backward pass
        num_classes = 10                        # number of cifar-10 dataset classes
        epochs =20              # number of forward and backward passes of all the training examples

        '''
            dataset contains the hyper parameters for loading data and the dataset:
                dataset = {
                    'batch_size': batch_size,
                    'num_classes': num_classes,
                    'epochs': epochs,
                    'x_train': x_train,
                    'x_test': x_test,
                    'y_train': y_train,
                    'y_test': y_test
                }
        '''
        from tensorflow.keras.datasets import cifar10

        (x_train, y_train), (x_test, y_test)=cifar10.load_data()
        
        print('Shape of x_train is {}'.format(x_train.shape))
        print('Shape of x_test is {}'.format(x_test.shape)) 
        print('Shape of y_train is {}'.format(y_train.shape))
        print('Shape of y_test is {}'.format(y_test.shape))

        from tensorflow.keras.utils import to_categorical

        # Normalizing
        x_train=x_train/255
        x_test=x_test/255
        
        #One hot encoding
        y_train_cat=to_categorical(y_train,10)
        y_test_cat=to_categorical(y_test,10)

    
        dataset = {
        'batch_size': batch_size,
        'num_classes': num_classes,
        'epochs': epochs,
        'x_train': x_train,
        'y_train': y_train_cat,
        'x_val': x_test,
        'y_val': y_test_cat
    }


        """model.fit(dataset['x_train'],
                                dataset['y_train'],
                                batch_size=dataset['batch_size'],
                                epochs=dataset['epochs'],
                                validation_data=(dataset['x_val'], dataset['y_val']),
                                shuffle=True)
                                
         history1=model1.fit(x_train,y_train_cat,epochs=20,validation_data=(x_test,y_test_cat))   
                                """



    
        #dataset = load_dataset(batch_size, num_classes, epochs)

        num_population = 10
        num_generation = 10
        num_offspring = 4

        # plot the best model obtained
        optCNN = genetic_algorithm(num_population, num_generation, num_offspring, dataset)

        # plot the training and validation loss and accuracy
        num_epoch = 20
        model = optCNN.build_model()
        model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
        history = model.fit(dataset['x_train'],
                            dataset['y_train'],
                            batch_size=dataset['batch_size'],
                            epochs=num_epoch,
                            validation_data=(dataset['x_test'], dataset['y_test']),
                            shuffle=True)
        optCNN.model = model                                        # model
        optCNN.fitness = history.history['val_loss'][-1]            # fitness

        print("\n\n-------------------------------------")
        print("The Final CNN has been evolved successfully in the individual", optCNN.name)
        print("-------------------------------------\n")
        daddy = load_network('parent_0')
        model = tf.keras.models.load_model('parent_0.h5')
        print("\n\n-------------------------------------")
        print("Summary of initial CNN")
        print(model.summary())
        print("Fitness of initial CNN:", daddy.fitness)

        print("\n\n-------------------------------------")
        print("Summary of evolved individual")
        print(optCNN.model.summary())
        print("Fitness of the evolved individual:", optCNN.fitness)
        print("-------------------------------------\n")

        plot_training(history)


if __name__ == '__main__':
    main()

Shape of x_train is (50000, 32, 32, 3)
Shape of x_test is (10000, 32, 32, 3)
Shape of y_train is (50000, 1)
Shape of y_test is (10000, 1)
Genetic Algorithm
----->Initializing Population
The block is:
[<__main__.Block object at 0x7ac3b7556d80>, <__main__.Block object at 0x7ac3977e4a80>, <__main__.Block object at 0x7ac3973cc2c0>]
Layer added successfully.
Layer added successfully.
Layer added successfully.
Layer added successfully.
Layer added successfully.
Layer added successfully.
Layer added successfully.
Model successfully built.
Type of model_final: <class 'keras.src.engine.sequential.Sequential'>
Training parent_0
Epoch 1/20


2025-02-26 20:49:32.926894: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_10/dropout_15/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


1563/1563 [==============================] - 8s 4ms/step - loss: 1.4557 - accuracy: 0.4823 - val_loss: 1.4302 - val_accuracy: 0.5009
Epoch 2/20
1563/1563 [==============================] - 6s 4ms/step - loss: 1.1594 - accuracy: 0.5956 - val_loss: 1.3921 - val_accuracy: 0.5124
Epoch 3/20
1563/1563 [==============================] - 6s 4ms/step - loss: 1.0609 - accuracy: 0.6321 - val_loss: 1.4985 - val_accuracy: 0.5279
Epoch 4/20
1563/1563 [==============================] - 6s 4ms/step - loss: 1.0072 - accuracy: 0.6560 - val_loss: 1.3451 - val_accuracy: 0.5617
Epoch 5/20
1563/1563 [==============================] - 6s 4ms/step - loss: 0.9672 - accuracy: 0.6718 - val_loss: 1.6466 - val_accuracy: 0.5202
Epoch 6/20
1563/1563 [==============================] - 6s 4ms/step - loss: 0.9502 - accuracy: 0.6827 - val_loss: 1.6873 - val_accuracy: 0.5443
Epoch 7/20
1563/1563 [==============================] - 6s 4ms/step - loss: 0.9470 - accuracy: 0.6854 - val_loss: 1.8869 - val_accuracy: 0.5507
Epo

KeyboardInterrupt: Interrupted by user

## To remove a folder
# Clear output folder
import os

def remove_folder_contents(folder):
    for the_file in os.listdir(folder):
        file_path = os.path.join(folder, the_file)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                remove_folder_contents(file_path)
                os.rmdir(file_path)
        except Exception as e:
            print(e)

folder_path = '/kaggle/working'
remove_folder_contents(folder_path)
os.rmdir(folder_path)

NameError: name 'model' is not defined

In [19]:
batch_size = 32                       # the number of training examples in one forward/backward pass
num_classes = 10                        # number of cifar-10 dataset classes
epochs =20   
dataset = load_dataset(batch_size, num_classes, epochs)

In [31]:
# Importing the necessary libraries, which we may or may not use. Its always good idea to import them befor (if you remember) else we can do it at any point of time no problem.
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPool2D,Dense,Flatten,Dropout,Input, AveragePooling2D, Activation,Conv2D, MaxPooling2D, BatchNormalization,Concatenate
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.keras import regularizers, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [41]:
from tensorflow.keras.datasets import cifar10

(x_train, y_train), (x_test, y_test)=cifar10.load_data()

print('Shape of x_train is {}'.format(x_train.shape))
print('Shape of x_test is {}'.format(x_test.shape)) 
print('Shape of y_train is {}'.format(y_train_cat.shape))
print('Shape of y_test is {}'.format(y_test_cat.shape))

from tensorflow.keras.utils import to_categorical

# Normalizing
x_train=x_train/255
x_test=x_test/255

#One hot encoding
y_train_cat=to_categorical(y_train,10)
y_test_cat=to_categorical(y_test,10)

Shape of x_train is (50000, 32, 32, 3)
Shape of x_test is (10000, 32, 32, 3)
Shape of y_train is (50000, 10)
Shape of y_test is (10000, 10, 10)


In [36]:
print('Shape of x_train is {}'.format(dataset['x_train'].shape))
print('Shape of x_test is {}'.format(dataset['x_test'].shape)) 
print('Shape of y_train is {}'.format(dataset['y_train'].shape))
print('Shape of y_test is {}'.format(dataset['y_test'].shape))

Shape of x_train is (50000, 32, 32, 3)
Shape of x_test is (10000, 32, 32, 3)
Shape of y_train is (50000, 10)
Shape of y_test is (10000, 10)


In [35]:
model1=Sequential()
model1.add(Conv2D(filters=32,kernel_size=(4,4),input_shape=(32,32,3),activation='relu'))
model1.add(MaxPool2D(pool_size=(2,2)))
model1.add(Conv2D(filters=32,kernel_size=(4,4),input_shape=(32,32,3),activation='relu'))
model1.add(MaxPool2D(pool_size=(2,2)))
model1.add(Flatten())
model1.add(Dense(256,activation='relu'))
model1.add(Dense(10,activation='softmax'))
model1.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
# training_steps = train_generator.samples//train_generator.batch_size
# validation_steps=test_generator.samples//test_generator.batch_size
# history=model1.fit_generator(train_generator,epochs=50,steps_per_epoch=training_steps,validation_data=test_generator,validation_steps=validation_steps,callbacks=[board])
history1=model1.fit(x_train,y_train_cat,epochs=20,validation_data=(x_test,y_test_cat))

Epoch 1/20
1563/1563 [==============================] - 8s 4ms/step - loss: 1.5148 - accuracy: 0.4520 - val_loss: 1.3265 - val_accuracy: 0.5361
Epoch 2/20
1563/1563 [==============================] - 6s 4ms/step - loss: 1.1916 - accuracy: 0.5775 - val_loss: 1.0876 - val_accuracy: 0.6223
Epoch 3/20
1563/1563 [==============================] - 6s 4ms/step - loss: 1.0279 - accuracy: 0.6405 - val_loss: 1.0639 - val_accuracy: 0.6334
Epoch 4/20
1563/1563 [==============================] - 6s 4ms/step - loss: 0.9205 - accuracy: 0.6792 - val_loss: 1.0122 - val_accuracy: 0.6513
Epoch 5/20
1563/1563 [==============================] - 6s 4ms/step - loss: 0.8350 - accuracy: 0.7075 - val_loss: 0.9852 - val_accuracy: 0.6588
Epoch 6/20
1563/1563 [==============================] - 6s 4ms/step - loss: 0.7655 - accuracy: 0.7327 - val_loss: 1.0358 - val_accuracy: 0.6510
Epoch 7/20
1563/1563 [==============================] - 6s 4ms/step - loss: 0.6995 - accuracy: 0.7551 - val_loss: 0.9811 - val_accuracy:

In [32]:
model2=Sequential()
model2.add(Conv2D(filters=32,kernel_size=(4,4),input_shape=(32,32,3),activation='relu'))
model2.add(MaxPool2D(pool_size=(2,2)))
model2.add(Dropout(0.25)) # Drop 25% of the units from the layer.
model2.add(Conv2D(filters=32,kernel_size=(4,4),input_shape=(32,32,3),activation='relu'))
model2.add(MaxPool2D(pool_size=(2,2)))
model2.add(Dropout(0.25))
model2.add(Flatten())
model2.add(Dense(256,activation='relu'))
model2.add(Dense(10,activation='softmax'))
model2.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
# training_steps = train_generator.samples//train_generator.batch_size
# validation_steps=test_generator.samples//test_generator.batch_size
# model2.fit_generator(train_generator,epochs=100,steps_per_epoch=training_steps,validation_data=test_generator,validation_steps=validation_steps,callbacks=[board])
history2=model2.fit(dataset['x_train'],dataset['y_train'],epochs=20,validation_data=(dataset['x_test'],dataset['y_test']))

Epoch 1/20


2025-02-26 20:30:00.571276: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_6/dropout_9/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


1563/1563 [==============================] - 9s 4ms/step - loss: 1.6039 - accuracy: 0.4139 - val_loss: 1.3765 - val_accuracy: 0.5180
Epoch 2/20
1563/1563 [==============================] - 6s 4ms/step - loss: 1.3016 - accuracy: 0.5333 - val_loss: 1.1508 - val_accuracy: 0.5989
Epoch 3/20
1563/1563 [==============================] - 6s 4ms/step - loss: 1.1598 - accuracy: 0.5903 - val_loss: 1.0574 - val_accuracy: 0.6274
Epoch 4/20
1563/1563 [==============================] - 6s 4ms/step - loss: 1.0719 - accuracy: 0.6216 - val_loss: 0.9879 - val_accuracy: 0.6543
Epoch 5/20
1563/1563 [==============================] - 6s 4ms/step - loss: 1.0032 - accuracy: 0.6480 - val_loss: 0.9901 - val_accuracy: 0.6570
Epoch 6/20
1563/1563 [==============================] - 6s 4ms/step - loss: 0.9419 - accuracy: 0.6687 - val_loss: 0.9262 - val_accuracy: 0.6822
Epoch 7/20
1563/1563 [==============================] - 6s 4ms/step - loss: 0.8977 - accuracy: 0.6853 - val_loss: 0.9059 - val_accuracy: 0.6837
Epo

In [37]:
import numpy as np
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

def load_dataset(batch_size, num_classes, epochs):
    # Load CIFAR-10 dataset
    (x_train_full, y_train_full), (x_test, y_test) = cifar10.load_data()

    # Normalize images (convert to float32 and scale to [0,1])
    x_train_full = x_train_full.astype('float32') / 255.0
    x_test = x_test.astype('float32') / 255.0


    print('Shape of x_train is {}'.format(x_train_full.shape))
    print('Shape of x_test is {}'.format(x_test.shape)) 
    print('Shape of y_train is {}'.format(y_train_full.shape))
    print('Shape of y_test is {}'.format(y_test.shape))
    
    # Convert labels to one-hot encoding
    y_train_full = to_categorical(y_train_full, num_classes)
    y_test = to_categorical(y_test, num_classes)

    print('Shape of y_train is {}'.format(y_train_full.shape))
    print('Shape of y_test is {}'.format(y_test.shape))
    
    # Split training data into 40,000 for training and 10,000 for validation
    x_train, x_val, y_train, y_val = train_test_split(
        x_train_full, y_train_full, test_size=0.2, random_state=42
    )

    dataset = {
        'batch_size': batch_size,
        'num_classes': num_classes,
        'epochs': epochs,
        'x_train': x_train,
        'y_train': y_train,
        'x_val': x_val,
        'y_val': y_val,
        'x_test': x_test,  
        'y_test': y_test
    }

    return dataset




In [38]:
batch_size = 32                       # the number of training examples in one forward/backward pass
num_classes = 10                        # number of cifar-10 dataset classes
epochs =20   
dataset = load_dataset(batch_size, num_classes, epochs)

Shape of x_train is (50000, 32, 32, 3)
Shape of x_test is (10000, 32, 32, 3)
Shape of y_train is (50000, 1)
Shape of y_test is (10000, 1)
Shape of y_train is (50000, 10)
Shape of y_test is (10000, 10)
